In [17]:
from bs4 import BeautifulSoup
from datetime import time
from datetime import datetime
from datetime import timedelta
from collections import Counter
import json
import requests
import csv
import pandas as pd
import numpy as np
import aiohttp
import asyncio
import concurrent.futures
import io
import re

In [18]:
Crossfit_Men = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/Analysis_Data/Crossfit_Men.csv')
Crossfit_Men.head()

/var/folders/xm/kc32pwjj5fj5xqf43yb4dz340000gn/T/ipykernel_14964/1936657856.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  Crossfit_Men = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/Analysis_Data/Crossfit_Men.csv')


,Athlete ID,Gender,Competitor Name,Country Name,Region Name,Affiliate Name,Height (in),Age (Years),Weight (lbs),Overall Rank,...,Grace (time),Grace (sec),Helen (time),Helen (sec),L1 Benchmark (time),L1 Benchmark (sec),Run 5k (time),Run 5k (sec),Sprint 400m (time),Sprint 400m (sec)
0,10942,M,Everett Sloan,Canada,North America East,CrossFit Bytown,67.0,45.0,200.0,68985.0,...,1:47,107.0,7:13,433.0,NaN,NaN,NaN,NaN,NaN,NaN
1,10926,M,Kevin Wood,Canada,North America East,CrossFit Moncton,75.0,43.0,196.0,35977.0,...,3:13,193.0,8:16,496.0,NaN,NaN,19:38,1178.0,1:00,60.0
2,10976,M,John Lazaro,United States,North America East,CrossFit Bridgewater,69.0,46.0,180.0,164180.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10977,M,Matthew Haynes,United States,North America West,CrossFit Westwood,71.0,31.0,180.0,165637.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10966,M,Nick DiMatteo,United States,North America East,CrossFit OTG,68.0,48.0,165.0,60027.0,...,NaN,NaN,9:01,541.0,NaN,NaN,21:55,1315.0,NaN,NaN


In [19]:
Crossfit_Women = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/Analysis_Data/Crossfit_Women.csv')
Crossfit_Women.head()

/var/folders/xm/kc32pwjj5fj5xqf43yb4dz340000gn/T/ipykernel_14964/2298068075.py:1: DtypeWarning: Columns (0,40) have mixed types. Specify dtype option on import or set low_memory=False.
  Crossfit_Women = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/Analysis_Data/Crossfit_Women.csv')


,Athlete ID,Gender,Competitor Name,Country Name,Region Name,Affiliate Name,Height (in),Age (Years),Weight (lbs),Overall Rank,...,Grace (time),Grace (sec),Helen (time),Helen (sec),L1 Benchmark (time),L1 Benchmark (sec),Run 5k (time),Run 5k (sec),Sprint 400m (time),Sprint 400m (sec)
0,1775.0,F,Daniella Verhoest,United States,North America East,CrossFit 908,62.0,35.0,145.0,6060.0,...,1:54,114.0,10:22,622.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1755.0,F,Robin Warnick,United States,North America East,CrossFit Terminus,66.0,47.0,145.0,70388.0,...,2:25,145.0,11:09,669.0,NaN,NaN,25:08,1508.0,1:20,80.0
2,1802.0,F,Melissa Kiel,United States,North America East,East Ormond Beach CrossFit,69.0,42.0,165.0,37421.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:23,83.0
3,1751.0,F,Philippa Robinson,Australia,Oceania,Ocean Fitness CrossFit,68.0,41.0,216.0,35925.0,...,1:52,112.0,9:11,551.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1659.0,F,Camas Barkemeyer,United States,North America West,CrossFit Kodiak Island,68.0,42.0,150.0,3944.0,...,NaN,NaN,NaN,NaN,NaN,NaN,23:51,1431.0,1:00,60.0


In [20]:
# Concatenate the two DataFrames vertically
Crossfit = pd.concat([Crossfit_Men, Crossfit_Women], ignore_index=True)



In [22]:
# Calculate BMI
Crossfit['BMI'] = (Crossfit['Weight (lbs)'] / (Crossfit['Height (in)'] ** 2)) * 703

# Remove the "BMI" column
bmi_column = Crossfit.pop('BMI')

# Find the index of the "Weight (lbs)" column
weight_index = Crossfit.columns.get_loc('Weight (lbs)')

# Insert the "BMI" column after the "Weight (lbs)" column
Crossfit.insert(weight_index + 1, 'BMI', bmi_column)

# Display the DataFrame with the updated column order
Crossfit.head()

,Athlete ID,Gender,Competitor Name,Country Name,Region Name,Affiliate Name,Height (in),Age (Years),Weight (lbs),BMI,...,Grace (time),Grace (sec),Helen (time),Helen (sec),L1 Benchmark (time),L1 Benchmark (sec),Run 5k (time),Run 5k (sec),Sprint 400m (time),Sprint 400m (sec)
0,10942,M,Everett Sloan,Canada,North America East,CrossFit Bytown,67.0,45.0,200.0,31.321007,...,1:47,107.0,7:13,433.0,NaN,NaN,NaN,NaN,NaN,NaN
1,10926,M,Kevin Wood,Canada,North America East,CrossFit Moncton,75.0,43.0,196.0,24.495644,...,3:13,193.0,8:16,496.0,NaN,NaN,19:38,1178.0,1:00,60.0
2,10976,M,John Lazaro,United States,North America East,CrossFit Bridgewater,69.0,46.0,180.0,26.578450,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10977,M,Matthew Haynes,United States,North America West,CrossFit Westwood,71.0,31.0,180.0,25.102162,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10966,M,Nick DiMatteo,United States,North America East,CrossFit OTG,68.0,48.0,165.0,25.085424,...,NaN,NaN,9:01,541.0,NaN,NaN,21:55,1315.0,NaN,NaN


In [23]:
# Export Crossfit to a CSV file
Crossfit.to_csv('Crossfit.csv', index=False)

In [24]:
Crossfit = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/Analysis_Data/Crossfit.csv')
Crossfit.head()

/var/folders/xm/kc32pwjj5fj5xqf43yb4dz340000gn/T/ipykernel_14964/595672981.py:1: DtypeWarning: Columns (0,41) have mixed types. Specify dtype option on import or set low_memory=False.
  Crossfit = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/Analysis_Data/Crossfit.csv')


,Athlete ID,Gender,Competitor Name,Country Name,Region Name,Affiliate Name,Height (in),Age (Years),Weight (lbs),BMI,...,Grace (time),Grace (sec),Helen (time),Helen (sec),L1 Benchmark (time),L1 Benchmark (sec),Run 5k (time),Run 5k (sec),Sprint 400m (time),Sprint 400m (sec)
0,10942,M,Everett Sloan,Canada,North America East,CrossFit Bytown,67.0,45.0,200.0,31.321007,...,1:47,107.0,7:13,433.0,NaN,NaN,NaN,NaN,NaN,NaN
1,10926,M,Kevin Wood,Canada,North America East,CrossFit Moncton,75.0,43.0,196.0,24.495644,...,3:13,193.0,8:16,496.0,NaN,NaN,19:38,1178.0,1:00,60.0
2,10976,M,John Lazaro,United States,North America East,CrossFit Bridgewater,69.0,46.0,180.0,26.578450,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10977,M,Matthew Haynes,United States,North America West,CrossFit Westwood,71.0,31.0,180.0,25.102162,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10966,M,Nick DiMatteo,United States,North America East,CrossFit OTG,68.0,48.0,165.0,25.085424,...,NaN,NaN,9:01,541.0,NaN,NaN,21:55,1315.0,NaN,NaN


In [ ]:
Crossfit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301218 entries, 0 to 301217
Data columns (total 46 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Athlete ID               301216 non-null  object 
 1   Gender                   301211 non-null  object 
 2   Competitor Name          301208 non-null  object 
 3   Country Name             300380 non-null  object 
 4   Region Name              301211 non-null  object 
 5   Affiliate Name           268843 non-null  object 
 6   Height (in)              136763 non-null  float64
 7   Age (Years)              301211 non-null  float64
 8   Weight (lbs)             132089 non-null  float64
 9   Overall Rank             301211 non-null  float64
 10  24.1 Rank                301211 non-null  float64
 11  24.2 Rank                301211 non-null  float64
 12  24.3 Rank                301211 non-null  float64
 13  2024 Rank                301215 non-null  float64
 14  2023